## Entering original data

In [41]:
import pandas as pd
df = pd.read_csv('data.csv')
df

,X1,X2,X3,X4,Y
0,0.213911,-0.098156,0.359414,-0.019163,13.077645
1,0.167625,0.362185,0.609916,0.963090,15.268998
2,1.359511,0.541254,0.722986,1.164716,43.068049
3,1.479617,1.168745,1.647872,1.915115,131.541140
4,1.864991,2.164680,1.606221,1.749175,294.034360
5,2.609254,2.624402,2.724956,2.796561,727.684370
6,2.739215,2.894874,2.775946,2.972926,880.554470
7,3.803973,3.460895,3.440592,3.150549,1620.382000
8,3.958295,4.054293,4.707514,4.073429,2724.805600
9,4.481045,4.159474,4.680320,4.489039,3148.405600


## Change of Variables

In [43]:
Z1 = (df['X1']).rename('Z1')
Z2 = ((df['X2']**2)*df['X1']).rename('Z2')
Z3 = ((df['X2']**2)*df['X3']).rename('Z3')
Z4 = (df['X1']*df['X3']*df['X4']).rename('Z4')
augdf = pd.concat([Z1, Z2, Z3, Z4, df['Y']], axis=1)
augdf.to_csv('augdata.csv', header=True, index=False)
augdf

,Z1,Z2,Z3,Z4,Y
0,0.213911,0.002061,0.003463,-0.001473,13.077645
1,0.167625,0.021989,0.080008,0.098464,15.268998
2,1.359511,0.398276,0.211803,1.144808,43.068049
3,1.479617,2.021105,2.250936,4.669472,131.541140
4,1.864991,8.739051,7.526495,5.239807,294.034360
5,2.609254,17.971206,18.768102,19.883838,727.684370
6,2.739215,22.955426,23.263238,22.605870,880.554470
7,3.803973,45.563195,41.210696,41.234131,1620.382000
8,3.958295,65.063637,77.378765,75.903171,2724.805600
9,4.481045,77.527574,80.975274,94.147382,3148.405600


## Multiple Regression Class

In [44]:
import numpy as np
from scipy.stats import f, t
class MultipleRegression():
    def __init__(self, augdf, alpha):
        self.X = augdf.to_numpy()[:, 0:4]
        self.X = np.insert(self.X, 0, np.ones((20,)), axis=1)
        self.y = df.to_numpy()[:, 4]
        self.y = self.y.reshape(-1, 1)
        self.alpha = alpha
        self.n = 20
        self.p = 5
        self.regDOF = self.p - 1
        self.errDOF = self.n - self.p
        self.totDOF = self.n - 1
        self.Bhat = np.dot(np.linalg.inv(np.dot(self.X.T, self.X)), np.dot(self.X.T, self.y))
        self.yhat = np.dot(self.X, self.Bhat)
        self.SSE = np.sum((self.y - self.yhat)**2)
        self.SST = np.sum((self.y - np.mean(self.y))**2)
        self.SSR = self.SST - self.SSE
        self.mSSR = self.SSR/self.regDOF
        self.mSSE = self.SSE/self.errDOF
        self.RMSE = np.sqrt(self.mSSE)
        self.Fstat = self.mSSR/self.mSSE
        self.p_Ftest = 2*(1-f.cdf(self.Fstat, self.regDOF, self.errDOF))
        self.Rsq = self.SSR/self.SST
        self.Rsq_adj = 1 - (self.n - 1)*(1 - self.Rsq)/(self.n - self.p)
        self.H0_Ftest = None
        self.CovBhat = [[]]
        self.SE = []
        self.Tstats = []
        self.pvalues = []
        self.sigs = []
        self.errors_at_alpha = []

    def ftest(self):
        if f.ppf(self.alpha/2, self.regDOF, self.errDOF) <= self.Fstat <= f.ppf(1-self.alpha/2, self.regDOF, self.errDOF):
            self.H0_Ftest = 1
            # print('Null Hypothesis is true')
            
        else:
            self.H0_Ftest = 0
            # print('Null Hypothesis Rejected')
        return self.H0_Ftest
    
    def estimate(self):
        if self.ftest() == 0:
            self.CovBhat = np.linalg.inv(np.dot(self.X.T, self.X))*self.mSSE
            for i in range(0,5):
                self.SE.append(np.sqrt(model.CovBhat[i][i]))
                self.Tstats.append(self.Bhat[i][0]/self.SE[i])
                self.pvalues.append(2*(1 - t.cdf(self.Tstats[i], self.errDOF)))
                self.errors_at_alpha.append(self.SE[i]*abs(t.ppf(self.alpha/2, self.errDOF)))

    def summary(self):
        self.estimate()
        self.sigs = ['Significant' if self.pvalues[i] < self.alpha else 'Not Significant' for i in range(0,len(self.pvalues))]
        results = pd.concat([pd.Series(np.reshape(model.Bhat, (self.p,)), name = 'Coefficients'), pd.Series(self.SE, name = 'SEs'), pd.Series(self.sigs, name = 'Significance'), pd.Series(self.errors_at_alpha, name = f'Errors at {round(self.alpha*100, 2)}% ')], axis=1)
        results.index = ['Intercept', 'Z1', 'Z2', 'Z3', 'Z4']
        return results

    def anova(self):
        C1 = pd.Series(np.array([self.regDOF, self.errDOF, self.totDOF]), name = 'DOF')
        C2 = pd.Series(np.array([self.SSR, self.SSE, self.SST]), name = 'Sum Sqs')
        C3 = pd.Series(np.array([self.mSSR, self.mSSE, '-']), name = 'Mean Sum Sqs')
        C4 = pd.Series(np.array([self.Fstat, '-', '-']), name = 'F-value')
        C5 = pd.Series([self.p_Ftest, '-', '-'], name = 'p-value')
        anova_table = pd.concat([C1, C2, C3, C4, C5], axis=1)
        anova_table.index = ['Residual', 'Error', 'Total']
        return anova_table

        
    

## Obtaining the Results

In [45]:
model = MultipleRegression(augdf, alpha=0.0145)
summary = model.summary()
summary

,Coefficients,SEs,Significance,Errors at 1.45%
Intercept,11.098233,0.597544,Significant,1.651052
Z1,8.432722,0.227710,Significant,0.629176
Z2,12.938975,0.021209,Significant,0.058603
Z3,12.125195,0.024339,Significant,0.067250
Z4,11.831251,0.015307,Significant,0.042294


## ANOVA Table

In [46]:
tbl = model.anova()
print(f"\n Rsq = {model.Rsq}    Rsq adjusted = {model.Rsq_adj}")
tbl


 Rsq = 0.999999991220029    Rsq adjusted = 0.9999999888787034


,DOF,Sum Sqs,Mean Sum Sqs,F-value,p-value
Residual,4,1.840998e+09,460249403.70589644,427108468.3563706,2.22045e-16
Error,15,1.616391e+01,1.077593721044823,-,-
Total,19,1.840998e+09,-,-,-
